# Load json velov

In [30]:
# Open Json velov dataset 
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
import folium
from folium.plugins import HeatMap
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import LineString
import requests
import io
import os 
import sys 

sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'..','..','..')))
from load_inputs.Lyon.pt.subway_in import get_trigram_correspondance

data_folder  = '../../../../../../data/rrochas/raw_data'
bike_json_path = 'VELO-LYON/locations/pvo_patrimoine_voirie.pvostationvelov.json'
subway_csv_path = 'keolis_data_2019-2020/ref_subway.csv'
# subway_lane_path = 'keolis_data_2019-2020/ref_lines.csv'
subway_lanes_path = 'keolis_data_2019-2020/metro_matching_TRI_FULL.CSV'




In [27]:
# Load Subway data
df_subway = pd.read_csv(os.path.join(data_folder, subway_csv_path),index_col =0)
subway_lanes = pd.read_csv(os.path.join(data_folder,subway_lanes_path),sep=';').rename(columns = {'LIGNE_A':'A', 'LIGNE_B':'B', 'LIGNE_C':'C', 'LIGNE_D':'D'})
# subway_lanes['max_line'] = subway_lanes[['A','B','C','D']].idxmax(1)   # Does not really work 
# laneA = list(set(subway_lanes[subway_lanes['max_line']=='A']['TRI_STA'].tolist()))
# laneB = list(set(subway_lanes[subway_lanes['max_line']=='B']['TRI_STA'].tolist()))
# laneC = list(set(subway_lanes[subway_lanes['max_line']=='C']['TRI_STA'].tolist()))
# laneD = list(set(subway_lanes[subway_lanes['max_line']=='D']['TRI_STA'].tolist()))

laneA =  ['PER','AMP','BEL', 'COR','HOT','FOC', 'MAS', 'CHA', 'REP','GRA','FLA','CUS', 'BON', 'SOI']
laneB = ['CHA','BRO','PAR','GUI', 'SAX',  'MAC', 'JAU','DEB', 'GER','OGA' ]
laneC = ['CUI', 'HEN','CRO','CPA','HOT']
laneD = ['VAI','VMY','GOR','FOU', 'JEA','BEL',  'GIL',  'SAX','GAR','SAN', 'LUM','BLA', 'LAE', 'MER', 'PRY','VEN']

df_subway_A = df_subway[df_subway['COD_TRG'].isin(laneA)].set_index('COD_TRG').reindex(laneA)
df_subway_B = df_subway[df_subway['COD_TRG'].isin(laneB)].set_index('COD_TRG').reindex(laneB)
df_subway_C = df_subway[df_subway['COD_TRG'].isin(laneC)].set_index('COD_TRG').reindex(laneC)
df_subway_D = df_subway[df_subway['COD_TRG'].isin(laneD)].set_index('COD_TRG').reindex(laneD)

colorA = 'red'
colorB = 'blue'
colorC = 'green'
colorD = 'yellow'

df_correspondance = get_trigram_correspondance()

### Bike

In [44]:
# Load Bike data
with open(os.path.join(data_folder, bike_json_path)) as f:
    bike_data = json.load(f)
df_bike = pd.json_normalize(bike_data['features'])
df_bike['geometry.coordinates'] = df_bike['geometry.coordinates'].apply(lambda x: (x[1],x[0]))
df_bike['station_id'] = df_bike['properties.idstation']
df_bike['geometry'] = df_bike['geometry.coordinates'].apply(lambda x: Point(x[1],x[0]))
df_bike = df_bike.set_geometry('geometry')
df_bike = df_bike.set_crs(epsg=4326)
df_bike = df_bike[['station_id','geometry']]

### Subway 

In [ ]:
gdf_subway_stations = gpd.GeoDataFrame()
gdf_subway_lines = gpd.GeoDataFrame()
for df_subway_i,lane_i in zip([df_subway_A,df_subway_B,df_subway_C,df_subway_D],['A','B','C','D']):
    df_subway_i.loc[:,'geometry'] = df_subway_i.apply(lambda row: Point(row['MEAN_X'],row['MEAN_Y']), axis=1)
    df_subway_i.loc[:,'lane'] = lane_i
    Linestring_i = LineString(df_subway_i['geometry'].tolist())
    gdf_subway_stations = pd.concat([gdf_subway_stations, gpd.GeoDataFrame(df_subway_i, geometry='geometry', crs='EPSG:4326')], ignore_index=True)
    gdf_subway_lines = pd.concat([gdf_subway_lines, gpd.GeoDataFrame({'lane':[lane_i],'geometry':[Linestring_i]}, geometry='geometry', crs='EPSG:4326')], ignore_index=True)

## Folium Map: 

In [56]:
m = folium.Map(location=[45.75, 4.85], zoom_start=12, tiles='CartoDB positron')
m = df_bike.explore(m=m, name = 'Bike-Sharing Stations', style_kwds={'radius':3, 'fillOpacity': 0.5,'color':'green', 'fillColor':'lightgreen'})
m = gdf_subway_stations.explore(m =m, column='lane', cmap = [colorA,colorB,colorC,colorD],  name= 'Subway Stations', style_kwds={'radius': 10, 'fillOpacity': 0.8})
m = gdf_subway_lines.explore(m=m, column = 'lane', cmap = [colorA,colorB,colorC,colorD], name= 'Subway Lanes', style_kwds={'weight': 5, 'opacity': 0.8})
folium.LayerControl().add_to(m)
m 

In [20]:
df_subway_C.COD_TRG.values

array(['CUI', 'HEN', 'HOT', 'CPA', 'CRO'], dtype=object)